# Stratified K-Fold

In [1]:
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import KFold #K-Fold
from sklearn.model_selection import StratifiedKFold #StratifiedKFold

## K-Fold의 학습레이블 분포

In [2]:
iris = load_iris()
label = iris.target
features = iris.data
df = pd.DataFrame(data = iris.data, columns = iris.feature_names)
df

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2
...,...,...,...,...
145,6.7,3.0,5.2,2.3
146,6.3,2.5,5.0,1.9
147,6.5,3.0,5.2,2.0
148,6.2,3.4,5.4,2.3


In [5]:
#iris 타겟값 확인
df['label'] = iris.target
# df['label'].value_counts()
df

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),label
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,2
146,6.3,2.5,5.0,1.9,2
147,6.5,3.0,5.2,2.0,2
148,6.2,3.4,5.4,2.3,2


In [9]:
kfold = KFold(n_splits = 3)

n_iter = 0 #교차검증 반복횟수를 나타내기 위한 변수
for train_index, test_index in kfold.split(df):
    n_iter += 1
    label_train = df['label'].iloc[train_index] #학습 레이블
    label_test = df['label'].iloc[test_index] #검증 레이블
    
    print(f"교차검증 : {n_iter}")
    print(f"학습레이블 분포 :\n{label_train.value_counts()}")
    print(f"검증레이블 분포 :\n{label_test.value_counts()}")

교차검증 : 1
학습레이블 분포 :
label
1    50
2    50
Name: count, dtype: int64
검증레이블 분포 :
label
0    50
Name: count, dtype: int64
교차검증 : 2
학습레이블 분포 :
label
0    50
2    50
Name: count, dtype: int64
검증레이블 분포 :
label
1    50
Name: count, dtype: int64
교차검증 : 3
학습레이블 분포 :
label
0    50
1    50
Name: count, dtype: int64
검증레이블 분포 :
label
2    50
Name: count, dtype: int64


## Stratified K-Fold

In [10]:
from sklearn.model_selection import StratifiedKFold

In [13]:
sfk = StratifiedKFold(n_splits = 3)
n_iter = 0
df
for train_index, test_index in sfk.split(df, df['label']):
    n_iter += 1
    label_train = df['label'].iloc[train_index]
    label_test = df['label'].iloc[test_index]
    
    print(f"교차검증 : {n_iter}")
    print(f"학습레이블 분포 :\n{label_train.value_counts()}")
    print(f"검증레이블 분포 :\n{label_test.value_counts()}")
    

교차검증 : 1
학습레이블 분포 :
label
2    34
0    33
1    33
Name: count, dtype: int64
검증레이블 분포 :
label
0    17
1    17
2    16
Name: count, dtype: int64
교차검증 : 2
학습레이블 분포 :
label
1    34
0    33
2    33
Name: count, dtype: int64
검증레이블 분포 :
label
0    17
2    17
1    16
Name: count, dtype: int64
교차검증 : 3
학습레이블 분포 :
label
0    34
1    33
2    33
Name: count, dtype: int64
검증레이블 분포 :
label
1    17
2    17
0    16
Name: count, dtype: int64


In [14]:
dt = DecisionTreeClassifier(random_state = 150)
sfk = StratifiedKFold(n_splits = 3)
n_iter = 0
cv = []

In [16]:
import numpy as np
from sklearn.metrics import accuracy_score

for train_index, test_index in sfk.split(features, label):
    X_train, X_test = features[train_index], features[test_index]
    y_train, y_test = label[train_index], label[test_index]
    
    #학습, 예측
    dt.fit(X_train, y_train)
    y_pred = dt.predict(X_test)
    
    n_iter += 1
    acc = np.round(accuracy_score(y_test, y_pred), 4)
    train_size = X_train.shape[0]
    test_size = X_test.shape[0]
    
    print(f"{n_iter} 교차 검증 정확도 : {acc}, 학습데이터 크기 : {train_size}, 검증데이터 크기 : {test_size}")
    print(f"{n_iter} 검증 세트 인덱스 : {test_index}")
    cv.append(acc)
    
print(cv)
print(f"평균 검증 정확도 : {np.mean(cv)}")

4 교차 검증 정확도 : 0.98, 학습데이터 크기 : 100, 검증데이터 크기 : 50
4 검증 세트 인덱스 : [  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  50
  51  52  53  54  55  56  57  58  59  60  61  62  63  64  65  66 100 101
 102 103 104 105 106 107 108 109 110 111 112 113 114 115]
5 교차 검증 정확도 : 0.94, 학습데이터 크기 : 100, 검증데이터 크기 : 50
5 검증 세트 인덱스 : [ 17  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  67
  68  69  70  71  72  73  74  75  76  77  78  79  80  81  82 116 117 118
 119 120 121 122 123 124 125 126 127 128 129 130 131 132]
6 교차 검증 정확도 : 0.98, 학습데이터 크기 : 100, 검증데이터 크기 : 50
6 검증 세트 인덱스 : [ 34  35  36  37  38  39  40  41  42  43  44  45  46  47  48  49  83  84
  85  86  87  88  89  90  91  92  93  94  95  96  97  98  99 133 134 135
 136 137 138 139 140 141 142 143 144 145 146 147 148 149]
[0.98, 0.94, 0.98, 0.98, 0.94, 0.98]
평균 검증 정확도 : 0.9666666666666668
